# Obtencion de sesiones


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import inline as inline

%matplotlib inline

pd.set_option('display.width', 400)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 200)

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

df = pd.read_csv('events_up_to_01062018.csv')
pd.options.mode.chained_assignment = None

/home/carito/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df= df.sort_values(by=["timestamp"])
df['just_date'] = df['timestamp'].dt.date
df['month'] = df['timestamp'].dt.month
df.reset_index(drop = True, inplace = True)

Me quedo con los datos del mes 5 

In [3]:
df_tiempos = df.loc[df["month"]==5]

Agrego la columna **'diff'** que me indica _diferencia de tiempo entre los distintos eventos temporales **por usuario**_

In [4]:
df_tiempos['diff'] = df_tiempos.groupby(['person'])['timestamp'].diff()

Paso la columna a string para detectar los valores "NaT" que indican el ingreso de un nuevo usuario 

In [5]:
df_tiempos['diff'] = df_tiempos['diff'].astype(str)
df_tiempos['new_user'] = df_tiempos['diff'] =='NaT'

Agrego la columna **'diff2'** que me indica la _diferencia entre eventos temporales_ (sin importar el usuario)

In [6]:
df_tiempos['diff2'] = (df_tiempos['timestamp'] - (df_tiempos['timestamp'].shift())) / np.timedelta64(1, 'h')

Reasigno la columna **'diff'** para volver a tener datos de tipo temporal y elimina los nulos

In [7]:
df_tiempos['diff'] = df_tiempos.groupby(['person'])['timestamp'].diff()

Lleno con 0s todos los nulos del data frame

In [8]:
df_tiempos=df_tiempos.fillna(0)

Identifico en la columna **'new_session_same_user'** los _comienzos de nuevas sesiones del mismo usuario_. Para ello se tiene en cuenta un umbral de 0.48 horas. Es decir, se considera que si el tiempo entre eventos es mayor a 0.48 horas, el evento corresponde a una nueva sesion del mismo usuario. En el informe adjunto se explica detalladamente la elección de este umbral.

In [9]:
df_tiempos['new_session_same_user'] = df_tiempos['diff2'] > 0.48

Identifico en la columna **'new_session_new_user'** los comienzos de _nuevas sesiones de un usuario nuevo._ Se tiene en cuenta el mismo umbral que antes. 

In [10]:
df_tiempos["diff"]=df_tiempos["diff"]/np.timedelta64(1, 'h')
df_tiempos["new_session_new_user"] = df_tiempos["diff"] > 0.48

Para contabilizar las sesiones nuevas (que pueden corresponder al mismo usuario o a un nuevo usuario) se realiza la operacion OR entre las columnas con datos booleanos calculadas previamente.

In [11]:
df_tiempos["new_session"]=df_tiempos["new_user"]|df_tiempos["new_session_same_user"]|df_tiempos["new_session_new_user"]

Enumero las sesiones para luego separarlas por agrupación

In [12]:
df_tiempos['sessionid'] = df_tiempos['new_session'].cumsum()

Como pusimos un umbral de 0.48 horas para la finalizacion de las sesiones, elimino de la columa **'diff'** los valores que superen ese valor (Me quedo solo con los False de la columna new_sesion). Estos valores se corresponderan al primer 'diff' de cada sesion, que debería ser siempre nulo.

In [13]:
df_tiempos=df_tiempos[df_tiempos.new_session==False]

Ahora si agrupo por **sessionid** y obtengo la información buscada

In [14]:
df_tiempos = df_tiempos.drop(["just_date","month","diff","new_user","diff","diff2","new_session_same_user","new_session_new_user","new_session"],axis=1)

In [15]:
df_tiempos.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,search_term,staticpage,campaign_source,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,sessionid
627763,2018-05-01 00:00:19,viewed product,7dab1178,0,2829.0,iPhone 6,Bom,16GB,Prateado,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
627766,2018-05-01 00:00:27,viewed product,2cbd12ad,0,6832.0,iPhone 6S,Muito Bom,16GB,Prateado,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
627767,2018-05-01 00:00:34,viewed product,4ba8900f,0,13249.0,Samsung Galaxy S8 Plus,Muito Bom,128GB,Preto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
627768,2018-05-01 00:00:36,viewed product,2cbd12ad,0,6831.0,iPhone 6S,Excelente,16GB,Prateado,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
627770,2018-05-01 00:00:38,visited site,754cd2b0,0,0.0,0,0,0,0,0,0,0,0,0,Paid,Returning,São Paulo,Sao Paulo,Brazil,Smartphone,360x640,Android 7,Chrome Mobile 66.0,5


In [16]:
df_tiempos.to_csv("datos_por_sesion.csv")